# Prepare and understand MLS stratospheric water vapour

We will use this as estimate for Hunga Tonga positive forcing.

To obtain the data, go to https://search.earthdata.nasa.gov/search?q=MLS%20H2O&fi=MLS&fl=3%2B-%2BGridded%2BObservations. Select the MLS/Aura Level 3 Monthly Binned Water Vapor (H2O) Mixing Ratio on Assorted Grids V005 (ML3MBH2O). Download this data to ../data/mls.

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as pl
import numpy as np
import glob

In [ ]:
data = np.ones((19*12, 11)) * np.nan

In [ ]:
for year in range(2004, 2023):
    nc = Dataset(glob.glob(f'../data/mls/MLS-Aura_L3MB-H2O_*_{year}.nc')[0])
    data[(year-2004)*12:(year-2004)*12+12, :] = nc.groups['H2O PressureZM'].variables['value'][:, 17:28, 15:30].mean(axis=2)
    plev = nc.groups['H2O PressureZM'].variables['lev'][17:28]
    nc.close()

In [ ]:
data

In [ ]:
X, Y = np.meshgrid(np.arange(2004+1/24, 2023, 1/12), plev)

In [ ]:
data[data==0] = np.nan

In [ ]:
pl.contourf(
    np.arange(2004+1/24, 2023, 1/12),
    plev[:],
    data.T
)
ax = pl.gca()
ax.set_ylim(ax.get_ylim()[::-1])
pl.colorbar()

In [ ]:
plev

In [ ]:
data[179]  # Dec 2018 NaN

## Need to conservatively regrid this to ERA5's plev / lon / lat

I regridded ERA5 to 2 degrees to run in SOCRATES, then we want H2O in terms of specific humidity (kg / kg) for 10, 20, 30 hPa with bounds (7.5, 15), (15, 25), (25, 40).